### Compute SNAP/ACE descriptors from coordinate files

Author: Noé Blassel

I used the package [FitSNAP](https://fitsnap.github.io/) which provides a convenient interface to compute SNAP and ACE descriptors, for the sake of exploration. Warning: getting it to work required a bit of tinkering.

To run, setup FitSNAP and LAMMPS for PACE according to the [instructions](https://fitsnap.github.io/Installation.html#lammps-pace-install), and modify the path in the cell below.

In [11]:
!pip install /home/nblassel/Documents/confs/chicago_24/working_group/FitSNAP

Defaulting to user installation because normal site-packages is not writeable
Processing /home/nblassel/Documents/confs/chicago_24/working_group/FitSNAP
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for fitsnap3: filename=fitsnap3-3.1.1.0-py3-none-any.whl size=247949 sha256=746c37a11407366529117dfb3e0e0c910219a4ef0f8208faa1a18cd0200dc022
  Stored in directory: /tmp/pip-ephem-wheel-cache-k6fazuyi/wheels/51/77/a1/aad0c1203a0ec993da8d447e9d121d62593db4fc8f8036aaa5
Successfully built fitsnap3
  Attempting uninstall: fitsnap3
    Found existing installation: fitsnap3 3.1.1.0
    Uninstalling fitsnap3-3.1.1.0:
      Successfully uninstalled fitsnap3-3.1.1.0


In [18]:
import pandas as pd

from ase import Atoms
from ase.io import read,write

from ase.visualize import view

from os import listdir
from os.path import join

import numpy as np

import re
from fitsnap3lib.fitsnap import FitSnap
from fitsnap3lib.scrapers.ase_funcs import ase_scraper

In [13]:
data_dir = "../data/transition_data_pt/"

def read_ase(filename):
    
    return read(
        filename=filename, 
        format="lammps-data", 
        Z_of_type={1: 78}, 
        style="atomic"
    )

In [14]:
files = listdir(data_dir)

df = pd.DataFrame(columns = ["initial_state","final_state","initial_conf","final_conf","initial_file","final_file"])

for f in files:
    m = re.match('(\d+)_(\d+)_([01])\.dat',f)
    if m is not None:
        state_ini, state_fin, is_final = map(int, m.groups())
        if is_final == 1:
            conf_fin = read_ase(join(data_dir,f))
            conf_ini = read_ase(join(data_dir,f"{state_ini}_{state_fin}_0.dat"))
        df.loc[len(df)]=[state_ini,state_fin,conf_ini,conf_fin,f"{state_ini}_{state_fin}_0.dat",f]

df

,initial_state,final_state,initial_conf,final_conf,initial_file,final_file
0,2458,2514,"(Atom('Pt', [16.891824119192098, 10.1507631789...","(Atom('Pt', [16.853061085379768, 10.1137640799...",2458_2514_0.dat,2458_2514_1.dat
1,2417,2418,"(Atom('Pt', [11.715215934019668, 16.3911847516...","(Atom('Pt', [11.821776976486966, 16.3676259690...",2417_2418_0.dat,2417_2418_1.dat
2,1023,1024,"(Atom('Pt', [19.207468506917255, 20.3295459034...","(Atom('Pt', [19.15307574312301, 20.30019765624...",1023_1024_0.dat,1023_1024_1.dat
3,868,869,"(Atom('Pt', [19.207468506917255, 20.3295459034...","(Atom('Pt', [19.15307574312301, 20.30019765624...",868_869_0.dat,868_869_0.dat
4,540,542,"(Atom('Pt', [19.207468506917255, 20.3295459034...","(Atom('Pt', [19.15307574312301, 20.30019765624...",540_542_0.dat,540_542_0.dat
...,...,...,...,...,...,...
6923,2115,2128,"(Atom('Pt', [19.713114789263727, 20.9530791355...","(Atom('Pt', [19.767766072487134, 20.8601510259...",2115_2128_0.dat,2115_2128_0.dat
6924,2166,1689,"(Atom('Pt', [19.3945175019235, 20.233189112181...","(Atom('Pt', [19.463021736568283, 20.2095783358...",2166_1689_0.dat,2166_1689_1.dat
6925,945,946,"(Atom('Pt', [19.3945175019235, 20.233189112181...","(Atom('Pt', [19.463021736568283, 20.2095783358...",945_946_0.dat,945_946_0.dat
6926,1723,1735,"(Atom('Pt', [8.403605399621133, 15.66192933852...","(Atom('Pt', [8.255799203538341, 15.73714546328...",1723_1735_0.dat,1723_1735_1.dat


In [15]:
ix = 100

view([df.initial_conf[ix],df.final_conf[ix]])

Here I just use the parameters given in the FitSNAP [tutorial](https://colab.research.google.com/github/FitSNAP/FitSNAP/blob/master/tutorial.ipynb)

In [17]:
sna_settings =\
{
"BISPECTRUM":
    {
    "numTypes": 1,
    "twojmax": 6,
    "rcutfac": 4.67637,
    "rfac0": 0.99363,
    "rmin0": 0.0,
    "wj": 1.0,
    "radelem": 0.5,
    "type": "Pt",
    "wselfallflag": 0,
    "bzeroflag": 1,
    "bikflag": 1
    },
"CALCULATOR":
    {
    "calculator": "LAMMPSSNAP",
    "energy": 1,
    "force": 0,
    "stress": 0,
    "per_atom_energy": 1
    },
"REFERENCE":
    {
    "units": "metal",
    "atom_style": "atomic",
    "pair_style": "zero 6.0",
    "pair_coeff": "* *"
    }
}

ace_settings = \
{
"ACE":
    {
    "numTypes": 1,
    "ranks": "1 2 3",
    "lmax":  "1 2 2",
    "nmax": "22 2 2",
    "nmaxbase": 22,
    "rcutfac": 4.604694451,
    "lambda": 3.5,
    "type": "Pt",
    "lmin": 0,
    "bzeroflag": 1,
    "bikflag": 1,
    "RPI_heuristic": "root_SO3_span"
    },
"CALCULATOR":
    {
    "calculator": "LAMMPSPACE",
    "energy": 1,
    "force": 0,
    "stress": 0,
    "per_atom_energy": 1
    },
"REFERENCE":
    {
    "units": "metal",
    "atom_style": "atomic",
    "pair_style": "zero 6.0",
    "pair_coeff": "* *"
    }
}

fs_sna = FitSnap(sna_settings, arglist=["--overwrite"])
fs_ace = FitSnap(ace_settings, arglist=["--overwrite"])

In [33]:
sna_ini = []
sna_fin = []

ace_ini = []
ace_fin = []

dat_ini = ase_scraper(list(df.initial_conf))
dat_fin = ase_scraper(list(df.final_conf))

Warning: long.

In [35]:
for i in range(len(df)):
    print(f'{i}/{len(df)}',end=" ")
    a,_,_ = fs_sna.calculator.process_single(dat_ini[i])
    sna_ini.append(a)

    a,_,_ = fs_sna.calculator.process_single(dat_fin[i])
    sna_fin.append(a)

    a,_,_ = fs_ace.calculator.process_single(dat_ini[i])
    ace_ini.append(a)

    a,_,_ = fs_ace.calculator.process_single(dat_fin[i])
    ace_fin.append(a)
    

0/6928 1/6928 2/6928 3/6928 4/6928 5/6928 6/6928 7/6928 8/6928 9/6928 10/6928 11/6928 12/6928 13/6928 14/6928 15/6928 16/6928 17/6928 18/6928 19/6928 20/6928 21/6928 22/6928 23/6928 24/6928 25/6928 26/6928 27/6928 28/6928 29/6928 30/6928 31/6928 32/6928 33/6928 34/6928 35/6928 36/6928 37/6928 38/6928 39/6928 40/6928 41/6928 42/6928 43/6928 44/6928 45/6928 46/6928 47/6928 48/6928 49/6928 50/6928 51/6928 52/6928 53/6928 54/6928 55/6928 56/6928 57/6928 58/6928 59/6928 60/6928 61/6928 62/6928 63/6928 64/6928 65/6928 66/6928 67/6928 68/6928 69/6928 70/6928 71/6928 72/6928 73/6928 74/6928 75/6928 76/6928 77/6928 78/6928 79/6928 80/6928 81/6928 82/6928 83/6928 84/6928 85/6928 86/6928 87/6928 88/6928 89/6928 90/6928 91/6928 92/6928 93/6928 94/6928 95/6928 96/6928 97/6928 98/6928 99/6928 100/6928 101/6928 102/6928 103/6928 104/6928 105/6928 106/6928 107/6928 108/6928 109/6928 110/6928 111/6928 112/6928 113/6928 114/6928 115/6928 116/6928 117/6928 118/6928 119/6928 120/6928 121/6928 122/6928 123

In [44]:
df["initial_snap"] = pd.Series(sna_ini,index=df.index)
df["final_snap"] = pd.Series(sna_fin,index=df.index)
df["initial_ace"] = pd.Series(ace_ini,index=df.index)
df["final_ace"] = pd.Series(ace_fin,index=df.index)

In [46]:
df.to_pickle("transitions_df.p")

Final pickled dataframe is ~1.5 G, can share upon request